In [ ]:
! pip3 install langgraph langsmith langchain langchain-community arxiv wikipedia

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
import langchain_community

In [6]:
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

### Arxiv and wikipedia tools

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=300)
arxiv_tools = ArxivQueryRun(api_wrapper=arxiv_wrapper)

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tools = WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [ ]:
print(
    wiki_tools.run("explain me the theory of relativity in brief")
)  # wiki_tools.run("explain me the theory of relativity in brief")

Page: Special relativity
Summary: In physics, the special theory of relativity, or special relativity for short, is a scientific theory of the relationship between space and time. In Albert Einstein's 1905 paper, 
"On the Electrodynamics of Moving Bodies", the theory is presented as being based on j


In [ ]:
print(
    arxiv_tools.run(
        "explain me the latest research papers on quantum computing in brief"
    )
)  # arxiv_tools.run("explain me the latest research papers on quantum computing in brief")

Published: 2025-04-07
Title: Quantum Computing: Vision and Challenges
Authors: Sukhpal Singh Gill, Oktay Cetinkaya, Stefano Marrone, Daniel Claudino, David Haunschild, Leon Schlote, Huaming Wu, Carlo Ottaviani, Xiaoyuan Liu, Sree Pragna Machupalli, Kamalpreet Kaur, Priyansh Arora, Ji Liu, Ahmed Faro


In [ ]:
tools = [wiki_tools]
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(state_schema=State)

In [20]:
import openai
from dotenv import load_dotenv
load_dotenv("../.env")
import os
openai_client = openai.OpenAI(api_key=os.getenv("OPENAI_API_TOKEN"), base_url=os.getenv("OPENAI_API_BASE"))
client = openai_client.chat.completions.create(
    messages=[{
        "role": "user",
        "content": "what is the capital of india and who is the prime minister of india, give full name of the prime minister?"
    }], model="gpt-4o", temperature=0.5)

print(client.choices[0].message.content)

The capital of India is New Delhi. As of my last update, the Prime Minister of India is Narendra Modi, whose full name is Narendra Damodardas Modi.


In [ ]:
import langchain_community
llm = langchain_commu